In [2]:
from torch.utils.data import Dataset, DataLoader

class ConditionalZiziDataset(Dataset):
    def __init__(self, input_dir):
        self.img_dir = input_dir + "test_img"
        self.pose_dir = input_dir + "test_openpose"
        self.img_files = os.listdir(self.img_dir)
        self.img_files.sort()
        self.preprocess = transforms.Compose(
            [
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5])
            ]
        )

    def __len__(self):
        return len(self.img_files)

    def _get_img(self, idx):
        img_dir = os.path.join(self.img_dir, self.img_files[idx])
        return self.preprocess(Image.open(img_dir).convert("RGB"))

    def _get_json(self, idx):
        input_split = os.path.splitext(self.img_files[idx])
        pose_json_name = input_split[0] + "_keypoints.json"
        json_dir = os.path.join(self.pose_dir, pose_json_name)
        with open(json_dir, 'r') as f:
            data = json.load(f)

        if 'people' not in data:
            print(f"people not found: {pose_json_name}")
            return torch.zeros(75)

        if len(data['people']) == 0:
            print(f"no people found: {pose_json_name}")
            return torch.zeros(75)

        if 'pose_keypoints_2d' not in data['people'][0]:
            print(f"no keypoints found: {pose_json_name}")
            return torch.zeros(75)
        
        return torch.tensor(data['people'][0]['pose_keypoints_2d'])
    
    def __getitem__(self, idx):
        if isinstance(idx, slice):
            # Get the start, stop, and step from the slice
            return [self[ii] for ii in range(*idx.indices(len(self)))]

        input_img = self._get_img(idx)
        json_data = self._get_json(idx)

        return {
            "images": input_img,
            "poses": json_data
        }

In [3]:
x = ConditionalZiziDataset("data/pink-cape-me")

TypeError: __init__() takes 2 positional arguments but 3 were given